Total_Turnover: 3422065.0586887286  
Total_PnL: 147777.01029799972  
Total_Commission: 123075.89073176196  
Total_Net_PnL: 24701.119566237743  

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('horses.csv')

In [3]:
df.head()

,race_number,saddle_number,win_fair_price,win_starting_price,winner
0,1,4,1.7353,1.7098,1
1,1,1,6.0313,6.0914,0
2,1,5,7.6923,7.5101,0
3,1,6,20.3325,20.4978,0
4,1,2,23.9991,23.4710,0


In [4]:
df.tail()

,race_number,saddle_number,win_fair_price,win_starting_price,winner
86643,10000,7,14.4772,13.8269,0
86644,10000,5,29.0062,29.8636,0
86645,10000,3,50.8005,48.9715,0
86646,10000,9,310.1959,305.6448,0
86647,10000,2,621.1930,601.6791,0


In [5]:
df.dtypes

race_number             int64
saddle_number           int64
win_fair_price        float64
win_starting_price    float64
winner                  int64
dtype: object

In [6]:
len(df)

86648

### Calculate overlay
As an example if the *win_starting_price* is the price that the bookmaker offers on the horse as it starts the race and if our own probability of the horse winning is the *win_fair_price* we calculate the value (overlay):  
`( (1 / win_fair_price )% multiplied by win_starting_price)) - 100%.`

In [7]:
estimated_prob_winning = (1 / df["win_fair_price"])
odds = df["win_starting_price"]
df["overlay"] = (estimated_prob_winning * odds) - 1
df

,race_number,saddle_number,win_fair_price,win_starting_price,winner,overlay
0,1,4,1.7353,1.7098,1,-0.014695
1,1,1,6.0313,6.0914,0,0.009965
2,1,5,7.6923,7.5101,0,-0.023686
3,1,6,20.3325,20.4978,0,0.008130
4,1,2,23.9991,23.4710,0,-0.022005
...,...,...,...,...,...,...
86643,10000,7,14.4772,13.8269,0,-0.044919
86644,10000,5,29.0062,29.8636,0,0.029559
86645,10000,3,50.8005,48.9715,0,-0.036004
86646,10000,9,310.1959,305.6448,0,-0.014672


### Kelly Criterion

The formula used to calculate the fraction for a lay bet is:
Full Lay Kelly = $ f = (q - p × (d - 1)) / (d - 1) $ where 
- f = fraction of the current bank to wager.  
- d = decimal betting odds.
- p = perceived probability of selection winning.
- q = perceived probability of selection losing, which is 1 – p.

The formula used to calculate the fraction for a back bet is:
Full Back Kelly = $ f = (p × (d – 1) – q) / (d – 1) $ where

- f = fraction of the current bank to wager.
- d = decimal betting odds.
- p = perceived probability of selection winning.
- q = perceived probability of selection losing, which is 1 – p.  

Betting full Kelly we can also calculate the stake as: `bankroll * (overlay/(odds - 1))` where overlay is the absolute value

In [8]:
overlay_is_null = df["overlay"].isnull()
print(overlay_is_null.value_counts())
overlay_is_not_null = df["overlay"].notnull()
print(overlay_is_not_null.value_counts())

False    86648
Name: overlay, dtype: int64
True    86648
Name: overlay, dtype: int64


In [9]:
df["side"] = ["lay" if x < 0 else "back" for x in df["overlay"]]
df.loc[df["overlay"] < 0, "kelly"] = 10000 * ((df["overlay"] * (-1)) / (odds - 1))
df.loc[df["overlay"] > 0, "kelly"] = 10000 * (df["overlay"] / (odds - 1))

### Turnover

In [10]:
df.loc[df["kelly"] > 2, "turnover"] = df["kelly"]
df.loc[df["kelly"] < 2, "turnover"] = 0
df

,race_number,saddle_number,win_fair_price,win_starting_price,winner,overlay,side,kelly,turnover
0,1,4,1.7353,1.7098,1,-0.014695,lay,207.028254,207.028254
1,1,1,6.0313,6.0914,0,0.009965,back,19.571600,19.571600
2,1,5,7.6923,7.5101,0,-0.023686,lay,36.383502,36.383502
3,1,6,20.3325,20.4978,0,0.008130,back,4.169620,4.169620
4,1,2,23.9991,23.4710,0,-0.022005,lay,9.792618,9.792618
...,...,...,...,...,...,...,...,...,...
86643,10000,7,14.4772,13.8269,0,-0.044919,lay,35.019301,35.019301
86644,10000,5,29.0062,29.8636,0,0.029559,back,10.240995,10.240995
86645,10000,3,50.8005,48.9715,0,-0.036004,lay,7.505203,7.505203
86646,10000,9,310.1959,305.6448,0,-0.014672,lay,0.481600,0.000000


### Expected Value

In [11]:
back_bets = df["side"] == "back"
df.loc[back_bets, "EV"] = (estimated_prob_winning * ((odds - 1) * df["kelly"])) - ((1 - estimated_prob_winning) * df["kelly"])
lay_bets = df["side"] == "lay"
df.loc[lay_bets, "EV"] = ((1 - estimated_prob_winning) * df["kelly"]) - (estimated_prob_winning * ((odds - 1) * df["kelly"]))
df

,race_number,saddle_number,win_fair_price,win_starting_price,winner,overlay,side,kelly,turnover,EV
0,1,4,1.7353,1.7098,1,-0.014695,lay,207.028254,207.028254,3.042252
1,1,1,6.0313,6.0914,0,0.009965,back,19.571600,19.571600,0.195025
2,1,5,7.6923,7.5101,0,-0.023686,lay,36.383502,36.383502,0.861780
3,1,6,20.3325,20.4978,0,0.008130,back,4.169620,4.169620,0.033898
4,1,2,23.9991,23.4710,0,-0.022005,lay,9.792618,9.792618,0.215486
...,...,...,...,...,...,...,...,...,...,...
86643,10000,7,14.4772,13.8269,0,-0.044919,lay,35.019301,35.019301,1.573029
86644,10000,5,29.0062,29.8636,0,0.029559,back,10.240995,10.240995,0.302716
86645,10000,3,50.8005,48.9715,0,-0.036004,lay,7.505203,7.505203,0.270214
86646,10000,9,310.1959,305.6448,0,-0.014672,lay,0.481600,0.000000,0.007066


### PnL

In [12]:
back_winners = (df["winner"] == 1) & (df["side"] == "back")
back_losers = (df["winner"] == 0) & (df["side"] == "back")
lay_winners = (df["winner"] == 0) & (df["side"] == "lay")
lay_losers = (df["winner"] == 1) & (df["side"] == "lay")

df.loc[back_winners, "PnL"] = df["turnover"] * ( odds - 1 ) 
df.loc[back_losers, "PnL"] = -df["turnover"]
df.loc[lay_winners, "PnL"] = df["turnover"]
df.loc[lay_losers, "PnL"] = -df["turnover"] * ( odds - 1 ) 
print(back_winners.value_counts() + back_losers.value_counts() + lay_winners.value_counts() + lay_losers.value_counts())
df

False    259944
True      86648
dtype: int64


,race_number,saddle_number,win_fair_price,win_starting_price,winner,overlay,side,kelly,turnover,EV,PnL
0,1,4,1.7353,1.7098,1,-0.014695,lay,207.028254,207.028254,3.042252,-146.948654
1,1,1,6.0313,6.0914,0,0.009965,back,19.571600,19.571600,0.195025,-19.571600
2,1,5,7.6923,7.5101,0,-0.023686,lay,36.383502,36.383502,0.861780,36.383502
3,1,6,20.3325,20.4978,0,0.008130,back,4.169620,4.169620,0.033898,-4.169620
4,1,2,23.9991,23.4710,0,-0.022005,lay,9.792618,9.792618,0.215486,9.792618
...,...,...,...,...,...,...,...,...,...,...,...
86643,10000,7,14.4772,13.8269,0,-0.044919,lay,35.019301,35.019301,1.573029,35.019301
86644,10000,5,29.0062,29.8636,0,0.029559,back,10.240995,10.240995,0.302716,-10.240995
86645,10000,3,50.8005,48.9715,0,-0.036004,lay,7.505203,7.505203,0.270214,7.505203
86646,10000,9,310.1959,305.6448,0,-0.014672,lay,0.481600,0.000000,0.007066,0.000000


### Commission, Net PnL & RoI

In [13]:
df["commission"] = [x * 0.05 if x > 0 else 0 for x in df["PnL"]]
df["Net PnL"] = df["PnL"] - df["commission"]
df.loc[df["turnover"] == 0, "RoI"] = 0
df.loc[df["turnover"] != 0, "RoI"] = df["Net PnL"] / df["turnover"]
df

,race_number,saddle_number,win_fair_price,win_starting_price,winner,overlay,side,kelly,turnover,EV,PnL,commission,Net PnL,RoI
0,1,4,1.7353,1.7098,1,-0.014695,lay,207.028254,207.028254,3.042252,-146.948654,0.000000,-146.948654,-0.7098
1,1,1,6.0313,6.0914,0,0.009965,back,19.571600,19.571600,0.195025,-19.571600,0.000000,-19.571600,-1.0000
2,1,5,7.6923,7.5101,0,-0.023686,lay,36.383502,36.383502,0.861780,36.383502,1.819175,34.564327,0.9500
3,1,6,20.3325,20.4978,0,0.008130,back,4.169620,4.169620,0.033898,-4.169620,0.000000,-4.169620,-1.0000
4,1,2,23.9991,23.4710,0,-0.022005,lay,9.792618,9.792618,0.215486,9.792618,0.489631,9.302987,0.9500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86643,10000,7,14.4772,13.8269,0,-0.044919,lay,35.019301,35.019301,1.573029,35.019301,1.750965,33.268336,0.9500
86644,10000,5,29.0062,29.8636,0,0.029559,back,10.240995,10.240995,0.302716,-10.240995,0.000000,-10.240995,-1.0000
86645,10000,3,50.8005,48.9715,0,-0.036004,lay,7.505203,7.505203,0.270214,7.505203,0.375260,7.129942,0.9500
86646,10000,9,310.1959,305.6448,0,-0.014672,lay,0.481600,0.000000,0.007066,0.000000,0.000000,0.000000,0.0000


### Total Turnover, Total EV, Total PnL, Total Net PnL, Total RoI


In [14]:
Total_Turnover = df["turnover"].sum()
Total_PnL = df["PnL"].sum()
Total_Commission = df["commission"].sum()
Total_Net_PnL = df["Net PnL"].sum()
print(f'Total_Turnover: {Total_Turnover}')
print(f'Total_PnL: {Total_PnL}')
print(f'Total_Commission: {Total_Commission}')
print(f'Total_Net_PnL: {Total_Net_PnL}')

Total_Turnover: 3422065.0586887286
Total_PnL: 147777.01029799972
Total_Commission: 123075.89073176196
Total_Net_PnL: 24701.119566237743


### Running a Monte Carlo simulation

In [15]:
import random

This is a very simplistic way of running a Monte Carlo simulation. I understand the idea is that you want to generate some random probability of winning and check how that impacts our PnL. What I have tried to do is:
- generate a random number between 0 and 1 for each bet
- if random number associated with each bet is less than the expected win probability, we have a winner  odds – 1. If it’s not, we assign a loss of -1.
determine whether each bet wins or loses in our simulation, we simply ask Excel whether the 

In [27]:
randomly_generated_results = {}
for i in range(10):
    df = pd.read_csv('horses.csv')
    random_numbers = [] 
    for price in df["win_fair_price"]:
          random_numbers.append(random.uniform(0, 1)) 
    df["random_prob_winning"] = random_numbers
    df["estimated_prob_winning"] = estimated_prob_winning

    # start the process again
    df["overlay"] = (estimated_prob_winning * odds) - 1
    df.loc[df["overlay"] < 0, "kelly"] = 10000 * ((df["overlay"] * (-1)) / (odds - 1))
    df.loc[df["overlay"] > 0, "kelly"] = 10000 * (df["overlay"] / (odds - 1))
    df.loc[df["kelly"] > 2, "turnover"] = df["kelly"]
    df.loc[df["kelly"] < 2, "turnover"] = 0
    
    df["side"] = ["lay" if x < 0 else "back" for x in df["overlay"]]
    back_bets = df["side"] == "back"
    lay_bets = df["side"] == "lay"
    
#     df.loc[back_bets & (estimated_prob_winning > df["random_prob_winning"]), "winner"] = 1
#     df.loc[back_bets & (estimated_prob_winning < df["random_prob_winning"]), "winner"] = 0
#     df.loc[lay_bets & (estimated_prob_winning < df["random_prob_winning"]), "winner"] = 1 
#     df.loc[lay_bets & (estimated_prob_winning > df["random_prob_winning"]), "winner"] = 0 
    
    df.loc[estimated_prob_winning > df["random_prob_winning"], "winner"] = 1
    df.loc[estimated_prob_winning < df["random_prob_winning"], "winner"] = 0
    
    back_winners = (df["winner"] == 1) & (df["side"] == "back")
    back_losers = (df["winner"] == 0) & (df["side"] == "back")
    lay_winners = (df["winner"] == 0) & (df["side"] == "lay")
    lay_losers = (df["winner"] == 1) & (df["side"] == "lay")
    
    df.loc[back_bets, "EV"] = (estimated_prob_winning * ((odds - 1) * df["kelly"])) - ((1 - estimated_prob_winning) * df["kelly"])
    df.loc[lay_bets, "EV"] = ((1 - estimated_prob_winning) * df["kelly"]) - (estimated_prob_winning * ((odds - 1) * df["kelly"]))

    df.loc[back_winners, "PnL"] = df["turnover"] * ( odds - 1 ) 
    df.loc[back_losers, "PnL"] = -df["turnover"]
    df.loc[lay_winners, "PnL"] = df["turnover"]
    df.loc[lay_losers, "PnL"] = -df["turnover"] * ( odds - 1 ) 
    df["commission"] = [x * 0.05 if x > 0 else 0 for x in df["PnL"]]
    df["Net PnL"] = df["PnL"] - df["commission"]
    df.loc[df["turnover"] == 0, "RoI"] = 0
    df.loc[df["turnover"] != 0, "RoI"] = df["Net PnL"] / df["turnover"]
    randomly_generated_results[f'Net_PnL_{i+1}'] = df["Net PnL"].sum()

print(randomly_generated_results)


{'Net_PnL_1': -7192.797015058297, 'Net_PnL_2': 24680.306875665974, 'Net_PnL_3': -1689.9218590893488, 'Net_PnL_4': -64458.79624991768, 'Net_PnL_5': 33121.80273947963, 'Net_PnL_6': 5298.878410693299, 'Net_PnL_7': 14785.727304242962, 'Net_PnL_8': 63862.10438124588, 'Net_PnL_9': 47641.453207229206, 'Net_PnL_10': -29043.155108646904}
